In [ ]:
# compare psse and pypsa

In [ ]:
import os
import wecgrid

wec_grid = wecgrid.Engine()

In [ ]:
wec_grid.case(f"{os.getcwd()}/../data/grid_models/IEEE_30_bus.raw")
wec_grid.load(["psse","pypsa"])